# Imports

In [1]:
import pandas as pd 
import numpy as np
#pyspark for analysis:
from pyspark.sql import SparkSession
import tensorflow as tf

In [2]:
SparkSession

pyspark.sql.session.SparkSession

In [3]:
spark = SparkSession.builder.master("local").appName("Py-SQL-analysis").getOrCreate()

24/11/19 14:29:41 WARN Utils: Your hostname, Admins-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.5.82 instead (on interface en1)
24/11/19 14:29:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 14:29:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Get and initially view Dataset

In [6]:
df = spark.read.csv("gym_members_exercise_tracking.csv", header=True)
#Let's view head(4), to see what the dataset we have in question
df.show(n=3)

+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
|Age|Gender|Weight (kg)|Height (m)|Max_BPM|Avg_BPM|Resting_BPM|Session_Duration (hours)|Calories_Burned|Workout_Type|Fat_Percentage|Water_Intake (liters)|Workout_Frequency (days/week)|Experience_Level|  BMI|
+---+------+-----------+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+
| 56|  Male|       88.3|      1.71|    180|    157|         60|                    1.69|         1313.0|        Yoga|          12.6|                  3.5|                            4|               3| 30.2|
| 46|Female|       74.9|      1.53|    179|    151|         66|                     1.3|          883.0|        HIIT|          33.9|                  2.1|              

In [22]:
#Save table to catalog
df.write.saveAsTable("health_data")
#Read catalog 
print(spark.catalog.listTables())

[Table(name='health_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]


In [28]:
temp_df = spark.sql("select _c1, _c2 from health_data limit 3")

In [32]:
temp_df.show()

ConnectionRefusedError: [Errno 61] Connection refused

As one can see, the columns are set arbitrarily.
Instead, we want to set the first row to the column names

In [30]:
temp_df.toPandas()

,_c1,_c2
0,Gender,Weight (kg)
1,Male,88.3
2,Female,74.9


24/11/19 02:18:27 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 911560 ms exceeds timeout 120000 ms
24/11/19 02:18:27 WARN SparkContext: Killing executors is not supported by current scheduler.
24/11/19 02:18:34 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc

In [13]:
row_1 = df.collect()[0]

In [14]:
row_1

Row(_c0='Age', _c1='Gender', _c2='Weight (kg)', _c3='Height (m)', _c4='Max_BPM', _c5='Avg_BPM', _c6='Resting_BPM', _c7='Session_Duration (hours)', _c8='Calories_Burned', _c9='Workout_Type', _c10='Fat_Percentage', _c11='Water_Intake (liters)', _c12='Workout_Frequency (days/week)', _c13='Experience_Level', _c14='BMI')